In [1]:
from joblib import Parallel, delayed
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# import SVC classifier
from sklearn.svm import SVC
# import metrics to compute accuracy
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [2]:
import os
from flask import Flask, flash, request, redirect, url_for, session
from werkzeug.utils import secure_filename
from flask_cors import CORS, cross_origin
import logging
from flask import Response
from flask import send_file

In [ ]:
# X_test = pd.read_csv('testData.csv')
# y_test = X_test['isAnomaly']
# final_data = test_data.drop(['isAnomaly', 'Unnamed: 0', 'device.deviceCategory', 'date', 'geoNetwork.country', 'trafficSource.keyword', 'hits', 'reason'], axis=1)

In [ ]:
# final_data = X_test.drop(['isAnomaly', 'Unnamed: 0.1','Unnamed: 0', 'device.deviceCategory', 'date', 'geoNetwork.country', 'trafficSource.keyword', 'hits', 'reason'], axis=1)
# final_data

In [ ]:
# y_pred = svm.predict(final_data)

In [ ]:
# X_test['isAnomaly'] = y_pred

In [ ]:
uploaded_df= ""
X_test=""
UPLOAD_FOLDER = 'D:/uploads'
ALLOWED_EXTENSIONS = {'csv'}
 
app = Flask(__name__, template_folder='templateFiles', static_folder='staticFiles')
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/upload_data',  methods=("POST", "GET"))
def uploadFile():
    if request.method == 'POST':
        # upload file flask
        global uploaded_df
        uploaded_df = request.files['uploaded-file']
 
        # Extracting uploaded data file name
        data_filename = secure_filename(uploaded_df.filename)
 
        # flask upload file to database (defined uploaded folder in static path)
        uploaded_df.save("./".join([data_filename]))
 
        # Storing uploaded file path in flask session
        session['uploaded_data_file_path'] = os.path.join(app.config['UPLOAD_FOLDER'], data_filename)
 
        return "Success", 200

@app.route('/process_data',  methods=["GET"])
def process_data():
    svm = joblib.load('svm.pkl')
    global X_test
    X_test = pd.read_csv('testData.csv')
#     X_test = uploaded_df
    final_data = X_test.drop(['isAnomaly', 'Unnamed: 0.1','Unnamed: 0', 'device.deviceCategory', 'date', 'geoNetwork.country', 'trafficSource.keyword', 'hits', 'reason'], axis=1)
    y_pred = svm.predict(final_data)
    X_test['isAnomaly'] = y_pred
    X_test.to_csv('./fullnfinal.csv')
    return "Success", 200

@app.route('/get_data',  methods=["GET"])
def get_data():
    csv_path = './fullnfinal.csv'
    return send_file(csv_path, as_attachment=True)

if __name__ == "__main__":
    app.secret_key = os.urandom(24)
    app.run(debug=True,host="0.0.0.0",use_reloader=False)

flask_cors.CORS(app, expose_headers='Authorization')

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.174:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Dec/2022 06:02:15] "GET / HTTP/1.1" 404 -
C:\Users\iftit\AppData\Local\Temp\ipykernel_6216\2692086273.py:31: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  X_test = pd.read_csv('testData.csv')
127.0.0.1 - - [05/Dec/2022 06:07:20] "GET /process_data HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2022 06:07:44] "GET /get_data HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2022 06:08:03] "GET /get_data HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2022 06:08:07] "GET /get_data HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2022 06:16:02] "GET / HTTP/1.1" 404 -
192.168.0.174 - - [05/Dec/2022 06:16:04] "GET / HTTP/1.1" 404 -
192.168.0.100 - - [05/Dec/2022 06:16:24] "GET / HTTP/1.1" 404 -
192.168.0.100 - - [05/Dec/2022 06:16:24] "GET /favicon.ico HTTP/1.1" 404 -
